In [ ]:
pip install requests pandas pillow

In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=954c8477425d0349fbc2769528ece117debeb5b6ad5c22bd90e49a1b0a60aec1
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia


In [22]:
import requests
import pandas as pd
from PIL import Image
import json
import base64
import io
import wikipedia
from wikipedia.exceptions import PageError, DisambiguationError, WikipediaException
from IPython.display import display, Image as DisplayImage
import hashlib

In [ ]:
# Загрузка базы данных из CSV
df = pd.read_csv("plants_watering.csv", names=[
    "scientific_name", "common_name", "family", "watering_frequency", "notes", "toxicity", "repotting", "light_requirements", "health"
], header=None)

In [32]:
# Функция для чтения изображения и кодирования в Base64
def image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def get_description_from_external_source(scientific_name):
    """Получаем описание из Wikipedia"""
    try:
        wikipedia.set_lang("ru")
        page = wikipedia.page(scientific_name, auto_suggest=False)
        return page.summary[:2000] + "..."  # Ограничиваем длину описания

    except PageError:
        return f"Описание для {scientific_name} не найдено в Wikipedia"
    except DisambiguationError as e:
        return f"Неоднозначный запрос: {e.options[0]}"
    except WikipediaException as e:
        return f"Ошибка Wikipedia: {str(e)}"
    except Exception as e:
        return f"Ошибка при запросе: {str(e)}"



def extract_plant_description(plant_data):
    """Извлекаем описание из первого предложения API"""
    try:
        if not plant_data.get("suggestions"):
            return "Описание недоступно"

        first_suggestion = plant_data["suggestions"][0]
        plant_name = first_suggestion.get("plant_details", {}).get("scientific_name", "")

        # Пробуем получить описание из разных источников
        return (
            first_suggestion.get("plant_details", {}).get("description")
            or get_description_from_external_source(plant_name)
            or "Описание отсутствует"
        )

    except Exception as e:
        print(f"Ошибка получения описания: {e}")
        return "Не удалось получить описание"


# Функция для отображения изображения
def display_image(image_data, title=""):
    if isinstance(image_data, str):  # Если это путь к файлу
        display(DisplayImage(filename=image_data, width=300))
    elif isinstance(image_data, bytes):  # Если это байтовые данные
        display(DisplayImage(data=image_data, width=300))
    print(title)

# Функция для определения растения через Plant.id API
def get_plant_info(image_path):
    api_key = "dJnePWJr2JzgMp8MYYqnl9nHDnZAdbNVwoySY89M6mfdqC6D0z"
    url = "https://api.plant.id/v2/identify"

    image_base64 = image_to_base64(image_path)
    data = {
        "images": [image_base64],
        "modifiers": ["crops_fast"],
        "plant_language": "ru",
        "upload_images": True
    }

    headers = {
        "Content-Type": "application/json",
        "Api-Key": api_key
    }

    response = requests.post(url, json=data, headers=headers)
    return response.json(), image_base64



# Функция для поиска информации о поливе и описания
def get_watering_info(plant_data):
    if "suggestions" in plant_data and len(plant_data["suggestions"]) > 0:
        plant = plant_data["suggestions"][0]

        # Извлекаем научное название и род (genus)
        scientific_name = plant.get("plant_details", {}).get("scientific_name", "")
        genus = ""
        if scientific_name:
            parts = scientific_name.split()
            if len(parts) >= 1:
                genus = parts[0].capitalize()

        # Обрабатываем обычное название из API
        plant_name = plant.get("plant_name", "")
        common_name_from_api = ""
        if isinstance(plant_name, dict):
            common_name_from_api = plant_name.get("common", "")
        elif isinstance(plant_name, str):
            common_name_from_api = plant_name

        # Поиск в базе данных
        row = df[
            (df["scientific_name"] == scientific_name) |
            (df["common_name"] == common_name_from_api)
        ]

        if not row.empty:
            common_name = row.iloc[0]["common_name"]
            description = extract_plant_description(plant_data)  # Описание из API
            return {
                "name": common_name or common_name_from_api,
                "scientific_name": scientific_name,
                "watering": row.iloc[0]["watering_frequency"],
                "notes": row.iloc[0]["notes"],
                "toxicity": row.iloc[0]["toxicity"],
                "repotting": row.iloc[0]["repotting"],
                "light_requirements": row.iloc[0]["light_requirements"],
                "health": row.iloc[0]["health"],
                "description": description
            }
        else:
            # Поиск по роду (genus)
            row = df[df["scientific_name"].str.startswith(genus)]
            if not row.empty:
                common_name = row.iloc[0]["common_name"]
                description = extract_plant_description(plant_data)
                return {
                    "name": common_name or common_name_from_api,
                    "scientific_name": scientific_name,
                    "watering": row.iloc[0]["watering_frequency"],
                    "notes": row.iloc[0]["notes"],
                    "toxicity": row.iloc[0]["toxicity"],
                    "repotting": row.iloc[0]["repotting"],
                    "light_requirements": row.iloc[0]["light_requirements"],
                    "health": row.iloc[0]["health"],
                    "description": description
                }
            else:
                # Поиск по семейству (family)
                family = plant.get("plant_family", "")
                row = df[df["family"] == family]
                if not row.empty:
                    common_name = row.iloc[0]["common_name"]
                    description = extract_plant_description(plant_data)
                    return {
                        "name": common_name or common_name_from_api,
                        "scientific_name": scientific_name,
                        "watering": row.iloc[0]["watering_frequency"],
                        "notes": row.iloc[0]["notes"],
                        "toxicity": row.iloc[0]["toxicity"],
                        "repotting": row.iloc[0]["repotting"],
                        "light_requirements": row.iloc[0]["light_requirements"],
                        "health": row.iloc[0]["health"],
                        "description": description
                    }
    return None


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
if __name__ == "__main__":
    image_path = "/content/drive/MyDrive/plant_test2.jpg"

    # Получаем информацию от Plant.id API
    plant_response, uploaded_image_base64 = get_plant_info(image_path)
    result = {}

    if plant_response:
        watering_info = get_watering_info(plant_response)
        if watering_info:
            # Формируем структуру результата
            result = {
                "name": watering_info.get("name", ""),
                "scientific_name": watering_info.get("scientific_name", ""),
                "description": watering_info.get("description", ""),
                "watering": watering_info.get("watering", ""),
                "notes": watering_info.get("notes", ""),
                "toxicity": watering_info.get("toxicity", ""),
                "repotting": watering_info.get("repotting", ""),
                "light_requirements": watering_info.get("light_requirements", ""),
                "health": watering_info.get("health", ""),
                "image_base64": uploaded_image_base64  # опционально
            }
        else:
            result["error"] = "Информация о растении не найдена"
    else:
        result["error"] = "Ошибка запроса к API"

    # Сохраняем результат в JSON файл
    with open("plant_info.json", "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=4)

    # Выводим результат в консоль
    print(json.dumps(result, ensure_ascii=False, indent=4))

{
    "name": "Фикус Benjamin",
    "scientific_name": "Ficus benjamina",
    "description": "Фи́кус бенджами́на (лат. Ficus benjamina) — вид растений из рода Фикус семейства Тутовые (Moraceae). Вечнозелёное дерево или кустарник, в дикой природе вырастает до 25 м в высоту. Распространён в Индии, Китае, Юго-Восточной Азии, на Филиппинах, на севере Австралии.\n\n...",
    "watering": "1–2 раза в неделю",
    "notes": "Не переливать. Поливать только при высыхании верхнего слоя почвы.",
    "toxicity": "Да",
    "repotting": "раз в 2–3 года",
    "light_requirements": "яркий свет/полутень",
    "health": "Болезнь/Паразиты не обнаружены. Растение здорово",
    "image_base64": "..."
}
